In [65]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import regex as re
import os
import seaborn as sns
from openpyxl import load_workbook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [66]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/CSF/Analysis/Code/DQA/')

<h1>
    Import raw data files
</h1>

In [67]:
literacy_raw = pd.read_excel("./2022_11_08_up_raw_literacy_cleaned.xlsx", index_col=[0])
numeracy_raw = pd.read_excel("./2022_11_08_up_raw_numeracy_cleaned.xlsx", index_col=[0])

In [68]:
def total_score(scores):
    total_score = 0
    for score in scores:
        if score == 1:
            total_score += 1
    return total_score

In [69]:
general_info =['tabletUserName', 'assessment_date', 'school_details.State_label', 'school_details.District_label', \
               'school_details.Block_label', 'school_details.School_label', 'school_details.UDISE_cd_label']

student_info = ['SI_std_name', 'student_age', 'student_gender']

up_lit_scores = literacy_raw[general_info + student_info].set_index(literacy_raw.index)
up_num_scores = numeracy_raw[general_info + student_info].set_index(numeracy_raw.index)

<h1 style="color:blue;">
    Literary Sub-task Score Calculations
</h1>

<h3>
    Literacy 1: Listening Comprehension
</h3>

In [70]:
# Calculate total score on listening comprehension sub-task
literacy1 = [col for col in literacy_raw.columns if re.search(r'literacy1_q\d$', col)]
                                                         
up_lit_scores.loc[:, 'literacy1_total'] = literacy_raw.apply(lambda x: total_score([x[col] for col in literacy1]), axis=1)

In [71]:
# Extract other responses to oral vocabulary questions
literacy1_or = [col for col in literacy_raw.columns if re.search(r'literacy1\S*or$', col)]
with pd.ExcelWriter('literacy1_other_responses.xlsx') as writer: 
    for col in literacy1_or:
        literacy_raw[col].value_counts().reset_index().rename(columns = {'index':"Child's response", col:'Frequency'}).to_excel(writer, sheet_name=col)

<h3>
    Literacy 2: Oral Vocabulary
</h3>

In [72]:
# Calculate total score on oral comprehension sub-task
literacy2 = [col for col in literacy_raw.columns if re.search(r'literacy2_q\d+$', col)]
                                                         
up_lit_scores.loc[:, 'literacy2_total'] = literacy_raw.apply(lambda x: total_score([x[col] for col in literacy2]), axis=1)

In [73]:
# Extract other responses to oral vocabulary questions
literacy2_or = [col for col in literacy_raw.columns if re.search(r'literacy2\S*or$', col)]
with pd.ExcelWriter('literacy2_other_responses.xlsx') as writer: 
    for col in literacy2_or:
        literacy_raw[col].value_counts().reset_index().rename(columns = {'index':"Child's response", col:'Frequency'}).to_excel(writer, sheet_name=col)

<h3>
    Literacy 3: Initial Sound Identification
</h3>

In [74]:
# Calculate total score on initial sound identification sub-task
literacy3 = [col for col in literacy_raw.columns if re.search(r'literacy3_q+', col)]
                                                         
up_lit_scores.loc[:, 'literacy3_total'] = literacy_raw.apply(lambda x: total_score([x[col] for col in literacy3]), axis=1)

<h3>
    Literacy 4: Letter Recognition (Untimed)
</h3>

In [75]:
literacy4_ut = [col for col in literacy_raw.columns if re.search(r'literacy4_ut_grid_\d*$', col)]
    
# Calculate total score on letter naming (untimed) sub-task
up_lit_scores.loc[:, 'literacy4_ut_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy4_ut]), axis=1)

<h3>
    Literacy 4: Letter Recognition (Timed)
</h3>

In [76]:
literacy4_tt =  [col for col in literacy_raw.columns if re.search(r'literacy4_tt_grid_\d*$', col)]

# Calculate total score on letter naming (timed) sub-task
up_lit_scores.loc[:, 'literacy4_tt_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy4_tt]), axis=1)
up_lit_scores.loc[:, 'literacy4_tt_time_taken'] = np.where(literacy_raw['literacy4_tt_grid.time_remaining']!='UNDEFINED',literacy_raw['literacy4_tt_grid.time_remaining'],60)
up_lit_scores['literacy4_tt_time_taken']= 60 - pd.to_numeric(up_lit_scores['literacy4_tt_time_taken'],errors='coerce').convert_dtypes()
up_lit_scores.loc[:,'literacy4_tt_cpm'] = np.where(np.logical_or(up_lit_scores['literacy4_tt_total']==0, up_lit_scores['literacy4_tt_time_taken']==0),0,round(60*up_lit_scores['literacy4_tt_total']/up_lit_scores['literacy4_tt_time_taken'],0))


<h3>
    Literacy 5: Familiar Words Reading (Untimed)
</h3>

In [77]:
literacy5_ut = [col for col in literacy_raw.columns if re.search(r'literacy5_ut_grid_\d*$', col)]
    
up_lit_scores.loc[:, 'literacy5_ut_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy5_ut]), axis=1)

<h3>
    Literacy 5: Familiar Words Reading (Timed)
</h3>

In [78]:
literacy5_tt = [col for col in literacy_raw.columns if re.search(r'literacy5_tt_grid_\d*$', col)]

up_lit_scores.loc[:, 'literacy5_tt_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy5_tt]), axis=1)
up_lit_scores.loc[:, 'literacy5_tt_time_taken'] = np.where(literacy_raw['literacy5_tt_grid.time_remaining']!='UNDEFINED',literacy_raw['literacy5_tt_grid.time_remaining'],60)
up_lit_scores['literacy5_tt_time_taken']= 60 - pd.to_numeric(up_lit_scores['literacy5_tt_time_taken'],errors='coerce').convert_dtypes()
up_lit_scores.loc[:,'literacy5_tt_cpm'] = np.where(np.logical_or(up_lit_scores['literacy5_tt_total']==0, up_lit_scores['literacy5_tt_time_taken']==0),0,round(60*up_lit_scores['literacy5_tt_total']/up_lit_scores['literacy5_tt_time_taken'],0))



<h3>
    Literacy 6: Non-word Reading
</h3>

In [79]:
literacy6 = [col for col in literacy_raw.columns if re.search(r'literacy6_tt_grid_\d*$', col)]

up_lit_scores.loc[:, 'literacy6_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy6]), axis=1)
up_lit_scores.loc[:, 'literacy6_tt_time_taken'] = np.where(literacy_raw['literacy6_tt_grid.time_remaining']!='UNDEFINED',literacy_raw['literacy6_tt_grid.time_remaining'],60)
up_lit_scores['literacy6_tt_time_taken']= 60 - pd.to_numeric(up_lit_scores['literacy6_tt_time_taken'],errors='coerce').convert_dtypes()
up_lit_scores.loc[:,'literacy6_tt_cpm'] = np.where(np.logical_or(up_lit_scores['literacy6_total']==0, up_lit_scores['literacy6_tt_time_taken']==0),0,round(60*up_lit_scores['literacy6_total']/up_lit_scores['literacy6_tt_time_taken'],0))


<h3>
    Literacy 7: Oral Reading Fluency (Timed)
</h3>

In [80]:
literacy7 = [col for col in literacy_raw.columns if re.search(r'literacy7_tt_grid_\d*$', col)]

up_lit_scores.loc[:, 'literacy7_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy7]), axis=1)
up_lit_scores.loc[:, 'literacy7_tt_time_taken'] = np.where(literacy_raw['literacy7_tt_grid.time_remaining']!='UNDEFINED',literacy_raw['literacy7_tt_grid.time_remaining'],60)
up_lit_scores['literacy7_tt_time_taken']= 60 - pd.to_numeric(up_lit_scores['literacy7_tt_time_taken'],errors='coerce').convert_dtypes()
up_lit_scores.loc[:,'literacy7_tt_cpm'] = np.where(np.logical_or(up_lit_scores['literacy7_total']==0, up_lit_scores['literacy7_tt_time_taken']==0),0,round(60*up_lit_scores['literacy7_total']/up_lit_scores['literacy7_tt_time_taken'],0))


<h3>
    Literacy 8: Reading Comprehension (Untimed)
</h3>

In [81]:
literacy8_reading = [col for col in literacy_raw.columns if re.search(r'literacy8_ut_grid_\d*$', col)]

literacy8_comprehension = [col for col in literacy_raw.columns if re.search(r'literacy8_ut_q\d*$', col)]

up_lit_scores.loc[:, 'literacy8_reading_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy8_reading]), axis=1)

up_lit_scores.loc[:, 'literacy8_comprehension_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy8_comprehension]), axis=1)

<h3>
    Literacy 9a: Dictation (Letters)
</h3>

In [82]:
literacy9a = [col for col in literacy_raw.columns if re.search(r'literacy9a_ut_grid_\d*$', col)]

up_lit_scores.loc[:, 'literacy9a_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy9a]), axis=1)

<h3>
    Literacy 9a: Dictation (Words)
</h3>

In [83]:
literacy9b = [col for col in literacy_raw.columns if re.search(r'literacy9b_ut_grid_\d*$', col)]

up_lit_scores.loc[:, 'literacy9b_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy9b]), axis=1)

In [84]:
up_lit_scores.to_excel(datetime.now().strftime("%Y_%m_%d")+"_up_literacy_total_scores.xlsx")

<h2>
    B. Numeracy Sub-tasks Data Cleaning
</h2>

<h3>
    Numeracy 1: Counting
</h3>

In [85]:
# Calculate total score on counting sub-task
numeracy1 = [col for col in numeracy_raw.columns if re.search(r'numeracy1_tt_grid_\d*$', col)]

up_num_scores.loc[:, 'numeracy1_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy1]), axis=1)
up_num_scores.loc[:, 'numeracy1_tt_time_taken'] = np.where(numeracy_raw['numeracy1_tt_grid.time_remaining']!='UNDEFINED',numeracy_raw['numeracy1_tt_grid.time_remaining'],60)
up_num_scores['numeracy1_tt_time_taken']= 60 - pd.to_numeric(up_num_scores['numeracy1_tt_time_taken'],errors='coerce').convert_dtypes()
up_num_scores.loc[:,'numeracy1_tt_cpm'] = np.where(np.logical_or(up_num_scores['numeracy1_total']==0, up_num_scores['numeracy1_tt_time_taken']==0),0,round(60*up_num_scores['numeracy1_total']/up_num_scores['numeracy1_tt_time_taken'],0))


<h3>
    Numeracy 2: Number Recognition (Untimed)
</h3>

In [86]:
numeracy2_ut = [col for col in numeracy_raw.columns if re.search(r'numeracy2_ut_grid_\d*$', col)]

up_num_scores.loc[:, 'numeracy2_ut_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy2_ut]), axis=1)

<h3>
    Numeracy 2: Number Recognition (Timed)
</h3>

In [87]:
numeracy2_tt = [col for col in numeracy_raw.columns if re.search(r'numeracy2_tt_grid_\d*$', col)]

up_num_scores.loc[:, 'numeracy2_tt_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy2_tt]), axis=1)
up_num_scores.loc[:, 'numeracy2_tt_time_taken'] = np.where(numeracy_raw['numeracy2_tt_grid.time_remaining']!='UNDEFINED',numeracy_raw['numeracy2_tt_grid.time_remaining'],60)
up_num_scores['numeracy2_tt_time_taken']= 60 - pd.to_numeric(up_num_scores['numeracy2_tt_time_taken'],errors='coerce').convert_dtypes()
up_num_scores.loc[:,'numeracy2_tt_cpm'] = np.where(np.logical_or(up_num_scores['numeracy2_tt_total']==0, up_num_scores['numeracy2_tt_time_taken']==0),0,round(60*up_num_scores['numeracy2_tt_total']/up_num_scores['numeracy2_tt_time_taken'],0))


<h3>
    Numeracy 3: Number Comparison
</h3>

In [88]:
numeracy3 = [col for col in numeracy_raw.columns if re.search(r'numeracy3\w*', col)]

up_num_scores.loc[:, 'numeracy3_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy3]), axis=1)

<h3>
    Numeracy 4: Counting in Bundles
</h3>

In [89]:
numeracy4 = [col for col in numeracy_raw.columns if re.search(r'numeracy4_ut_q\d*$', col)]

up_num_scores.loc[:, 'numeracy4_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy4]), axis=1)

<h3>
    Numeracy 5: Missing Numbers
</h3>

In [90]:
numeracy5 = [col for col in numeracy_raw.columns if re.search(r'numeracy5_ut_q\d*$', col)]

up_num_scores.loc[:, 'numeracy5_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy5]), axis=1)

<h3>
    Numeracy 6: Addition
</h3>

In [91]:
numeracy6 = [col for col in numeracy_raw.columns if re.search(r'numeracy6_ut_q\d*$', col)]

up_num_scores.loc[:, 'numeracy6_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy6]), axis=1)

<h3>
    Numeracy 7: Subtraction
</h3>

In [92]:
numeracy7 = [col for col in numeracy_raw.columns if re.search(r'numeracy7_ut_q\d*$', col)]

up_num_scores.loc[:, 'numeracy7_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy7]), axis=1)

<h3>
    Numeracy 8: Word Problems
</h3>

In [93]:
numeracy8 = [col for col in numeracy_raw.columns if re.search(r'numeracy8_ut_q\d*$', col)]

up_num_scores.loc[:, 'numeracy8_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy8]), axis=1)

<h3>
    Numeracy 9a: Shape Recognition (Circle)
</h3>

In [94]:
numeracy9a = [col for col in numeracy_raw.columns if re.search(r'numeracy9a_ut_grid_\d$', col)]

def total_score_9a(scores):
    total_score_9a = 0
    if scores[0] == '1' and scores[1] == '1' and scores[2] == '0' and scores[3] == '1' and scores[4] == '1' and scores[5] == '1' and scores[6] == '1' and scores[7] == '0':
            total_score_9a += 1 
    return total_score_9a
    
up_num_scores.loc[:, 'numeracy9a_total'] = numeracy_raw.apply(lambda x: total_score_9a([x[score] for score in numeracy9a]), axis=1)

<h3>
    Numeracy 9b: Shape Recognition (Rectangle)
</h3>

In [95]:
numeracy9b = [col for col in numeracy_raw.columns if re.search(r'numeracy9b_ut_grid_\d$', col)]
    
def total_score_9b(scores):
    total_score_9b = 0
    if scores[0] == '1' and scores[1] == '0' and scores[2] == '1' and scores[3] == '1' and scores[4] == '0' and scores[5] == '1' and scores[6] == '1' and scores[7] == '0':
            total_score_9b += 1 
    return total_score_9b   
    
up_num_scores.loc[:, 'numeracy9b_total'] = numeracy_raw.apply(lambda x: total_score_9b([x[score] for score in numeracy9b]), axis=1)

In [96]:
up_num_scores.to_excel(datetime.now().strftime("%Y_%m_%d")+"_up_numeracy_total_scores.xlsx")